# Counting cards

Let's do a few experiments with a virtual deck of cards to tackle some problems that don't have closed form solutions and that would be impractical to do manually. 

We'll start with a relatively simple model and then make things more flexible and sophisticated. Along the way, we'll revisit the combinatoric functions and illustrate the use of the Python `dictionary`.

Beyond the concrete examples, we also consider some broader questions regarding the representation of our model and the tradeoffs that affect performance, flexibility and programming difficulty.

## Hands that sum to *n* with no face cards

How many ways are there to deal a hand of five cards from a standard 52-card deck such that the cards sum to a target value and the hand contains no face cards (jack, queen, king, ace)? 

At first glance, this looks like the well known combinatoric problem of partitioning a set of $n$ indistibguishable items into $k$ distinguishable bins, which has a simple closed form solution. Going deeper, we see that there are some important differences. For example, we have the constraints that card values are limited to two through ten and no value can appear more than four times - if our target is 35, this excludes solutions based on sets of card values such as (1, 1, 1, 1, 31) and (7, 7, 7, 7, 7).

This really does look like a case where we need a computational solution.

Since we can omit the face cards and only need the card values, we can represent our simplified deck using a list where the integers two through ten each appear four times. We'll then use the `combinations()` function to generate all five-card hands and count the number of hands that sum to our target.

In [ ]:
cards_simple = [2, 3, 4, 5, 6, 7, 8, 9, 10] * 4
print(cards_simple)

### Will this really work?

Excellent question! The `combinations()` function does not allow for the repeated selection of elements. It seems that we would need to use `combinations_with_replacement()`. But allowing replacement isn't quite right either since it would allow us to generate forbidden hands such as (2, 2, 2, 2, 2).

So how do we generate (2, 2, 2, 2, 3), (2, 2, 2, 2, 4), ... (2, 2, 2, 2, 10) while avoiding (2, 2, 2, 2, 2)? The answer lies in how the `combinations()` function works. Rather than selecting elements from our list based on value, it selects based on position. The 1st, 2nd, 3rd, and 4th instances of '2' in our list are all distinct elements.

Let's give this a try. Afterwards, we'll confirm using a more realistic representation of a deck of cards that we got the right answer.

In [ ]:
from itertools import product, permutations, combinations, combinations_with_replacement

In [ ]:
%%time
target = 35 # Set our target value
ncards = 5  # Size of hand

count = 0
for hand in combinations(cards_simple, ncards):
    if sum(hand) == target:
        count += 1
        
print(count)

We don't want to print out all the hands, but let's peek at the first few to convince ourselves that things look reasonable.

In [ ]:
%%time
target = 35
ncards = 5  # Size of hand

count = 0
for hand in combinations(cards_simple, ncards):
    if sum(hand) == target:
        count += 1
        print(hand)
    if count >= 10:
        break

The output is a little confusing since the hand (2, 3, 10, 10, 10) appears three times, but they really are distinct and corespond to choosing the three tens from different subsets of the suits (e.g. clubs, diamonds and hearts vs. clubs, diamonds and spades).

## Generalizing to hands that sum to *n*, face cards allowed

Let's generalize to hands that include face cards, using the standard values of 10 for jacks, queens and kings and 11 for aces. Our simple representation of the deck still works since we don't care about any property of the card except its value.

In [ ]:
# The 2nd, 3rd and 4th '10' correspond to jack, queen, king
cards_simple = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 11] * 4
print(cards_simple)

In [ ]:
%%time
target = 35 # Set our target value
ncards = 5  # Size of hand

count = 0
for hand in combinations(cards_simple, ncards):
    if sum(hand) == target:
        count += 1
        
print(count)

## Creating a more sophisticated deck of cards

Up to this point, we've asked questions that only depend on the values for the cards. If we want to ask more general questions, we'll need a realistic representation of the deck. One way, illustrated below, is to create a list of cards and a pair of dictionaries that associate values and suits with the cards.

+ Abbreviate clubs, diamonds, hearts, spades as C, D, H, S
+ Abbreviate jack, queen, king, ace as J, Q, K, A
+ Create card name by concatenating rank and suit, for example
    + two of clubs $\rightarrow$ 2C
    + king of hearts $\rightarrow$ KH

This wasn't the only way to represent the deck. Another approach might have been to use Python's `namedtuple` type, but we wanted to stick with something that should already be familiar.

In [ ]:
card_values = {'2C':2,  '3C':3,  '4C':4,  '5C':5,  '6C':6,  '7C':7, '8C':8, 
               '9C':9, '10C':10, 'JC':10, 'QC':10, 'KC':10, 'AC':11,
               '2D':2,  '3D':3,  '4D':4,  '5D':5,  '6D':6,  '7D':7, '8D':8, 
               '9D':9, '10D':10, 'JD':10, 'QD':10, 'KD':10, 'AD':11,
               '2H':2,  '3H':3,  '4H':4,  '5H':5,  '6H':6,  '7H':7, '8H':8, 
               '9H':9, '10H':10, 'JH':10, 'QH':10, 'KH':10, 'AH':11,
               '2S':2,  '3S':3,  '4S':4,  '5S':5,  '6S':6,  '7S':7, '8S':8, 
               '9S':9, '10S':10, 'JS':10, 'QS':10, 'KS':10, 'AS':11}

card_suits = {'2C':'C',  '3C':'C', '4C':'C', '5C':'C', '6C':'C', '7C':'C', '8C':'C', 
              '9C':'C', '10C':'C', 'JC':'C', 'QC':'C', 'KC':'C', 'AC':'C',
              '2D':'D',  '3D':'D', '4D':'D', '5D':'D', '6D':'D', '7D':'D', '8D':'D', 
              '9D':'D', '10D':'D', 'JD':'D', 'QD':'D', 'KD':'D', 'AD':'D',
              '2H':'H',  '3H':'H', '4H':'H', '5H':'H', '6H':'H', '7H':'H', '8H':'H', 
              '9H':'H', '10H':'H', 'JH':'H', 'QH':'H', 'KH':'H', 'AH':'H',
              '2S':'S',  '3S':'S', '4S':'S', '5S':'S', '6S':'S', '7S':'S', '8S':'S', 
              '9S':'S', '10S':'S', 'JS':'S', 'QS':'S', 'KS':'S', 'AS':'S'}

cards = card_values.keys()

### Revisiting hands that sum to *n*, face cards allowed

Starting with the code that we used earlier, we make a few modifications so that we iterate over the cards in a hand and accumulate the sum of values. If we did everything correctly and used the same target value, we should get the same answer. Code has also been modified to print out the first few hands.

In [ ]:
%%time
target = 35 # Set our target value
ncards = 5  # Size of hand
print_hands = 15

count = 0
for hand in combinations(cards, ncards):
    handsum = 0
    for card in hand:
        handsum += card_values[card]
    if handsum == target:
        count += 1
        if count <= print_hands:
            print(hand)
        
print()
print(count)

### Revisiting hands that sum to *n* with no face cards

Going back to our original problem, but using the more realistic deck, we need to make a few more changes to exclude the face cards.

In [ ]:
%%time 
target = 35
ncards = 5  # Size of hand
print_hands = 15

face_cards = ['JC', 'JD', 'JH', 'JS', 'QC', 'QD', 'QH', 'QS', 
              'KC', 'KD', 'KH', 'KS', 'AC', 'AD', 'AH', 'AS']

count = 0
for hand in combinations(cards, ncards):
    handsum = 0
    for card in hand:
        if card in (face_cards):
            handsum = -1
            break
        else:
            handsum += card_values[card]
    if handsum == target:
        count += 1
        if count <= print_hands:
            print(hand)
        
print()
print(count)

### Hands that sum to *n* with restrictions on suit

We didn't need the more realistic deck of cards to answer our original questions - number of hands that sum to *n*, with or without face cards. Of course, if we wanted to inspect the hands, it's informative to see the full card names rather than just the values. ('2C', '3C', '10C', '10D', '10H') is more useful than (2, 3, 10, 10, 10)

Let's now look at a problem that considers suit, by adding the restriction that the hand must contain cards from at least *m* suits. We do this by adding the suit of each card in the hand to a set (thereby, avoiding having a suit counted multiple times) and then testing both the sum of values and number of suits before incrementing the count.

In [ ]:
%%time
target = 35    # Set our target value
ncards = 5  # Size of hand
min_suits = 4  # Set the minimum number of suits in hand
print_hands = 15

count = 0
for hand in combinations(cards, ncards):
    handsum = 0
    suits = set()
    for card in hand:
        handsum += card_values[card]
        suits.add(card_suits[card])
    if handsum == target and len(suits) >= min_suits:
        count += 1
        if count <= print_hands:
            print(hand)
        
print()
print(count)

## Lessons learned

Using a more realistic representation of the deck of cards allows us to answer more questions, but it comes with a cost. Not only is the code more complex, but for problems that can be solved using the simpler representation, the time to solution is longer.

Our recommendation is to use the simplest representation that suits your needs. If you were already given a more complex representation of your system, consider generating a simpler representation from it.

## Exercises

(1) Go back to a few of the examples and look at the run time for smaller or larger hands (don't go too large since compute time will take too long).

(2) Starting with the full representation for the deck of cards, generate a simplified representation that that can be used to solve the "Hands that sum to *n* with no face cards" problem. Hint: use the dictionary `.values()` method and then the `del()` function to remove unwanted cards.

(3) Ignoring suit, use the `combinations_with_replacement()` function to count "hands that sum to *n*". Compare with the results you got when we used the restriction imposed by a deck of cards. Note that you can operate on lists where each item appears only once (e.g. \[2, 3, 4, 5, 6, 7, 8, 9, 10\])